In [332]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re
import socket

na_values = ['?', '', 'nope']
products = pd.read_csv('products.csv', sep=';', na_values = na_values)
products.head()


,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,NaN,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


In [333]:
products.shape

(200, 5)

In [334]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
username      200 non-null object
ip_address    199 non-null object
product       200 non-null object
price         200 non-null object
infos         200 non-null object
dtypes: object(5)
memory usage: 7.9+ KB


In [335]:
products[products['username'].isna() == True]
products[products['ip_address'].isna() == True]
products = products.dropna(subset=['ip_address'])

In [336]:
products['price'] = products['price'].str.split(' ').str[0]

In [337]:
def test_ip(addr):
    try:
        socket.inet_aton(str(addr))
        return True;
    except socket.error:
        return False;
products['ip_validate'] = products['ip_address'].apply(test_ip)

In [338]:
def test_requete(url):
    requete = requests.get(url)
    if requete:
        currency = requete.json()['currency_code']
        return currency;
    else:
        return None;

In [339]:
products['url'] = "https://api.ipgeolocationapi.com/geolocate/"
products['currency'] = (products['url'][products['ip_validate'] == True] + products['ip_address'][products['ip_validate'] == True]).apply(test_requete)

In [340]:
products.head()

,username,ip_address,product,price,infos,ip_validate,url,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,False,https://api.ipgeolocationapi.com/geolocate/,NaN
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,True,https://api.ipgeolocationapi.com/geolocate/,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,True,https://api.ipgeolocationapi.com/geolocate/,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",True,https://api.ipgeolocationapi.com/geolocate/,JPY
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,True,https://api.ipgeolocationapi.com/geolocate/,None


In [341]:
products['currency'].unique()

array([nan, None, 'USD', 'JPY', 'MXN', 'CNY', 'EUR', 'AUD', 'KRW', 'NGN',
       'BRL', 'PEN', 'ZAR', 'GBP', 'BYN', 'CHF', 'RUB', 'CAD', 'INR',
       'MYR', 'VND', 'TWD', 'DKK', 'IDR', 'BGN', 'IRR', 'AOA', 'KZT',
       'TRY', 'TND', 'RON'], dtype=object)

In [342]:
products['currency'].isna()
products = products.dropna(subset=['currency'])
products.head(22)

,username,ip_address,product,price,infos,ip_validate,url,currency
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,True,https://api.ipgeolocationapi.com/geolocate/,USD
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",True,https://api.ipgeolocationapi.com/geolocate/,JPY
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,True,https://api.ipgeolocationapi.com/geolocate/,MXN
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",True,https://api.ipgeolocationapi.com/geolocate/,MXN
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",True,https://api.ipgeolocationapi.com/geolocate/,USD
10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,"Contains peanut, sugar, egg and fish",True,https://api.ipgeolocationapi.com/geolocate/,USD
11,jdolbeyb,17.105.113.72,Wine - Ice Wine,679.57,"Ingredients: mustard, sugar and milk",True,https://api.ipgeolocationapi.com/geolocate/,USD
12,chaylandsc,208.239.186.79,Chicken Thigh - Bone Out,725.88,Ingredients: gluten and sugar,True,https://api.ipgeolocationapi.com/geolocate/,USD
13,mvidelerd,116.18.56.20,Trout Rainbow Whole,178.58,May contain peanut,True,https://api.ipgeolocationapi.com/geolocate/,CNY
14,jmulvine,87.198.88.207,Bread Crumbs - Panko,150.89,"Ingredients: gluten, soja, sugar and milk",True,https://api.ipgeolocationapi.com/geolocate/,EUR


In [343]:
def convert_to_euro(price, currency):
    url = "https://api.exchangeratesapi.io/latest"
    if currency == 'EUR':
        return price
    
    elif currency == 'NGN':
        price_eur = round(float(price)/402.48, 2)
        return price_eur
    
    elif currency == 'PEN':
        price_eur = round(float(price)/3.72, 2)
        return price_eur 
        
    elif currency == 'BYN':
        price_eur = round(float(price)/2.27, 2)
        return price_eur 
    
    elif currency == 'VND':
        price_eur = round(float(price)/25838.49, 2)
        return price_eur
        
    elif currency == 'TWD':
        price_eur = round(float(price)/34.06, 2)
        return price_eur
        
    elif currency == 'IRR':
        price_eur = round(float(price)/46883.5, 2)
        return price_eur
        
    elif currency == 'AOA':
        price_eur = round(float(price)/501.89, 2)
        return price_eur
        
    elif currency == 'KZT':
        price_eur = round(float(price)/434.34, 2)
        return price_eur
     
    elif currency == 'TND':
        price_eur = round(float(price)/3.15, 2)
        return price_eur
        
    else:
        devise = requests.get(url).json()['rates'][currency]
        price_eur = round(float(price)/float(devise), 2)
        return price_eur

products['price_eur'] = products.apply(lambda x: convert_to_euro(x['price'], x['currency']), axis=1)

In [344]:
products.drop(columns=['ip_validate', 'url']).head()

,username,ip_address,product,price,infos,currency,price_eur
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,314.6
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JPY,7.86
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.24
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MXN,4.14
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",USD,773.81


In [346]:
#products['infos'].unique()
products['infos'] = products.infos.str.replace(':', ' ')
products['infos'] = products.infos.str.replace(',', '')
products['sugar'] = products.infos.str.get_dummies(sep=' ').sugar
products['egg'] = products.infos.str.get_dummies(sep=' ').egg
products['milk'] = products.infos.str.get_dummies(sep=' ').milk
products['fish'] = products.infos.str.get_dummies(sep=' ').fish
products['gluten'] = products.infos.str.get_dummies(sep=' ').gluten
products['soja'] = products.infos.str.get_dummies(sep=' ').soja
products['peanut'] = products.infos.str.get_dummies(sep=' ').peanut
products['mustard'] = products.infos.str.get_dummies(sep=' ').mustard
products = products.drop(columns=['ip_validate', 'url'])
products.head()

,username,ip_address,product,price,infos,currency,price_eur,sugar,egg,milk,fish,gluten,soja,peanut,mustard
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,USD,314.6,0,0,0,0,1,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,May contain sugar egg and fish,JPY,7.86,1,1,0,1,0,0,0,0
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MXN,5.24,1,0,0,0,0,0,0,0
8,epridham8,187.129.113.105,Dried Figs,88.05,Ingredients sugar milk and fish,MXN,4.14,1,0,1,1,0,0,0,0
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,May contain peanut sugar milk and fish,USD,773.81,1,0,1,1,0,0,1,0
